In [13]:
import math
import numpy as np 
import scipy as sp

# ativando modo livro/literal...
def gauss_seidel1(a, b, x0, tol, maxiter=1000):
    n = len(b)
    x = x0[:]
    k = 0
    while k < maxiter:
        for i in range(n):
            s = 0.0
            for j in range(n):
                if j < i:
                    s = s + a[i][j]*x[j]
                elif j > i:
                    s = s + a[i][j]*x0[j]
            # calcula o novo valor 
            x[i] = (b[i] - s)/a[i][i]
        # no livro isso (incremento) está no passo/posição errada
        k = k + 1
        # no critério de convergência o livro fala uma coisa no texto (norma inf)
        # mas faz outra (norma 2) no algoritmo 7.2 (passo 4)
        if all(abs(x[i] - x0[i]) < tol for i in range(n)):
            break
        for i in range(n):
            x0[i] = x[i] 
    return k, x


# ativando modo tolo...
def gauss_seidel2(a, b, x0, tol, maxiter=1000):
    n = len(b)
    x = x0[:]
    k = 0
    while k < maxiter:
        k = k + 1
        for i in range(n):
            s = 0.0
            for j in range(n):
                if j == i:
                    continue
                s = s + a[i][j]*x[j]
            # guarda o valor antigo antes de atualizar
            x0[i] = x[i]
            # agora atualizamos o valor 
            x[i] = (b[i] - s)/a[i][i]
        # no critério de convergência o livro fala uma coisa no texto (norma inf)
        # mas faz outra (norma 2) no algoritmo 7.2 (passo 4)
        if all(abs(x[i] - x0[i]) < tol for i in range(n)):
            break
    return k, x


# ativando modo esperto (eu acho)...
def gauss_seidel3(a, b, x0, tol, maxiter=1000):
    n = len(b)
    x_old = x0.copy()
    x = x0.copy()
    k = 0
    for k in range(1, int(maxiter)+1):
        for i in range(n):
            # jeito mais "pythônico" de somar condicionalmente :-)
            s = sum(a[i][j]*x[j] for j in range(n) if j != i)
            # guarda o valor antigo antes de atualizar
            x_old[i] = x[i]
            # agora atualizamos o valor 
            x[i] = (b[i] - s)/a[i,i]
        # usando critério de convergência melhor (erro percentual)
        if all(abs(x[i] - x_old[i])/abs(x[i]) < tol for i in range(n)):
            return k, x
    return k, x


# ativando modo IA (solução do chatgpt)...
def gauss_seidel_chatgpt(A, b, x0, tolerance, max_iterations=1000):
    n = len(A)
    x = x0.copy()
    iteration = 0
    for iteration in range(1, int(max_iterations)+1):
        x_new = x.copy()
        for i in range(n):
            s = np.dot(A[i, :i], x_new[:i]) + np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s) / A[i, i]
        if np.allclose(x, x_new, rtol=tolerance):
            return iteration, x_new
        x = x_new
    return iteration, x

# define no problema/exemplo:
#a = [[10, -1, 2, 0], [-1, 11, -1, 3], [2, -1, 10, -1], [0, 3, -1, 8]]
#b = [6, 25, -11, 15]
#n = len(b)

n = 3
a = np.random.rand(n,n)
b = np.random.rand(n,)

# tolerância 
tol = 1e-3

x0 = [0]*n
print(gauss_seidel1(a, b, x0, tol))

# como a rotina usa/reaproveita a variável x0 
# temos que reinicializar os valores 
x0 = [0]*n
print(gauss_seidel2(a, b, x0, tol))

x0 = [0]*n
print(gauss_seidel3(a, b, x0, tol))

x0 = [0]*n
print(gauss_seidel_chatgpt(a, b, x0, tol))

print('numpy', np.linalg.solve(a, b))

print('scipy', sp.linalg.solve(a, b))

(55, [-0.8920917350693124, 2.8071462032478927, -0.35659340708848203])
(55, [-0.8920917350693124, 2.8071462032478927, -0.35659340708848203])
(50, [-0.889136280293271, 2.8013797523989283, -0.35550969776238406])
(50, [-0.889136280293271, 2.8013797523989283, -0.35550969776238406])
numpy [-0.89628582  2.81532937 -0.3581313 ]
scipy [-0.89628582  2.81532937 -0.3581313 ]


In [17]:
%%timeit
x0 = [0]*n
# livro (método mais tolo ainda)
gauss_seidel1(a, b, x0, n, tol)

5.63 µs ± 34.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [18]:
%%timeit
x0 = [0]*n
# implementação tola
gauss_seidel2(a, b, x0, n, tol)

5.63 µs ± 29.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [19]:
%%timeit
x0 = [0]*n
# implementação esperta (esperança é a última que morre)
gauss_seidel3(a, b, x0, n, tol)

652 ns ± 5.55 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [20]:
%%timeit
x0 = [0]*n
# chatgpt
gauss_seidel_chatgpt(a, b, x0, n, tol)

473 ns ± 4.26 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
%%timeit 
x = np.linalg.solve(a, b)

4.39 µs ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
%%timeit
x = sp.linalg.solve(a, b)

17.5 µs ± 88.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
